# P&D Scalper

#### ------------------- Description -------------------------------------------

Search for volatily, then enter the trade and do a trailing stop.

...

Respect the Pump

...

Raw code to modify it easily / understand it easily


#### ----------------------- Notebook parts ------------------------------------

1. API key setting
2. Le algorithme (trailing stop, scanning, combining the two)
3. Testing (new data / old data)
4. Optimizing

#### ------------------------------------------------------------------------------

## 0. Import lib

In [87]:
import binance as b #to get the quotes (get only)
#from binance.client import Client #to send orders
import numpy as np #numpy 
import time as t #to use the function sleep

## 1. API KEY SETTING

In [1]:
#api_key = 'MmGNjK8dN5HqJP6gXNndnK7R7BZJMbY0xsRvLXvEWkIpyqVIZq4lfz8TGfb3Hxg' #example key 
#api_secret = 'mp7XgL9BySagqwEc5AIAIE9s0TtBCweCTA9Rz32b9UVTi8gAYE0jmBeLMsmT1N' #example secret key

#client = Client(api_key, api_secret)

#No used in the following, but should be used to send orders

## 2. LE ALGORITHME

### 2.1 Le Trailing Stop

Setting "empty" variable and the percentage of loss, commition, etc. 
Things important to modify are pointed with "<------------------"

In [173]:
end=0 #to start the loop, empty variable

index_n="ETHUSDT" #to test the loop, here with ETHUSDT <------------------------------
start=float(b.prices()[index_n]) #getting the price of ETHUSDT with b.prices() as a string and then transforming into float
#add send order here

risk=0.02 #risk in percent / here is 0.02% not 2% <------------------------------
net_risk=(100-risk)/100 #resulting risk to compute stoploss price
stoploss=net_risk*start #getting the price for the stoploss

trail=start-stoploss #trailling stop value (difference between price and stoploss)

high=start #to start the loop
price_to_retain=start #to the start the loop

com=0.001 #comission in percentage divided by 100 / here it is 0.1% <------------------------------

While loop to do the trailing stop for an order and then compute the result of the transaction taking comission in account

In [174]:
while price_to_retain>0:  #will always be above 0, can also use while True
        
    if high<float(b.prices()[index_n]): #if price higher than high, set new high from price
        high=float(b.prices()[index_n]) ; print("new high :", high)
        stop=high-trail #computing the new stop
        
    if float(b.prices()[index_n])<high-trail: #if price under high-trail, close the trade
        end=float(b.prices()[index_n]); print("order closed")
        #add close order here
        
    if float(b.prices()[index_n])<stoploss: #if price under stoploss price, close the trade
        end=float(b.prices()[index_n]) ; print("order closed")
        
    t.sleep(1) #price refreshing rate, in seconds
    
    price_to_retain=float(b.prices()[index_n]) #actualizing price to retain
    
    print('high:', high, 'price: ', price_to_retain, ' stop :', stoploss)
    
    if end>0: #breaking the loop and computing the result
        result=end*(1-com)-start*(1+com) #comission to simulate the spread
        print('You won =', result)
        break

high: 1636.55 price:  1637.55  stop : 1635.862822
new high : 1637.74
high: 1637.74 price:  1640.0  stop : 1637.41269
new high : 1640.85
order closed
high: 1640.85 price:  1640.81  stop : 1640.52269
You won = 0.862760000000435


## 2.2 Le Scanning

Do le scanning of the market to find the most volatile crypto.

### 2.2.1 Getting the names of each crypto

(sketchy part, doesn't work anymore if the order of crypto change, need to find a better way)

In [177]:
list_=b.prices() #get the prices
data = list(list_.items()) #transform the dictionnary to a list
an_array = np.array(data) #transform the list to an array
name_list=[item[0] for item in an_array] #get the first item of each subarray

### 2.2.2 Scanning loop

Setting the parameters

In [ ]:
time=10 #time step between each prices data
percent=1.03 #percent of price 

Loop

In [184]:
price=b.prices() #getting the price
arr_old_0=np.array(list(price.items())) #tranforming the price dictionnary into an array
arr_old_1=[float(item[1]) for item in arr_old_0] #getting only the price from the array, setting the starting prices 
print("Loop started :)")

while True:
    
    print("Searching....")
    
    t.sleep(time) #sleeping the time set in the parameters
    
    price_new=b.prices() #getting new price
    arr_new_0=np.array(list(price_new.items())) #transforming the prices as before
    arr_new_1=[float(item[1]) for item in arr_new_0] #idem
    
    percent_table=np.divide(arr_new_1,arr_old_1) #getting the percent difference 
    
    
    max_percent=np.max(percent_table) #getting the max of the prices
    index=np.where(percent_table == max_percent) #getting the position of the max to get the name
    index_name=name_list[index[0][0]] #finding the name from the index of the max
    print(max_percent*100-100, ' percent for ',index_name)
    
    arr_old_1=arr_new_1 #setting the new prices to compare 

Loop started :)
Searching....
7.692307692307708  percent for  TROYBTC
Searching....
2.0  percent for  VIBEBTC
Searching....
2.631578947368425  percent for  CDTBTC
Searching....
4.761904761904773  percent for  STMXBTC
Searching....
16.66666666666667  percent for  TNBBTC
Searching....


KeyboardInterrupt: 

## 2.3 Le combining 

Combining scanning and buying / closing order with trailing stop

Parameters (percent threshold)

In [193]:
percent = 1.05 #the price change of 5%, then buy

Parameters for the trading loop

In [197]:
risk=0.02 #risk in percent / here is 0.02% not 2% <------------------------------
net_risk=(100-risk)/100 #resulting risk to compute stoploss price

com=0.001 #comission in percentage divided by 100 / here it is 0.1% <------------------------------

Loop

In [200]:
price=b.prices() #getting price
arr_old_0=np.array(list(price.items())) #same as before
arr_old_1=[float(item[1]) for item in arr_old_0] #same as before

print("Loop started :)")

while True:
    
    while True:
    
        print("Searching....")

        t.sleep(time) #sleeping the time set in the parameters

        price_new=b.prices() #getting new price
        arr_new_0=np.array(list(price_new.items())) #transforming the prices as before
        arr_new_1=[float(item[1]) for item in arr_new_0] #idem

        percent_table=np.divide(arr_new_1,arr_old_1) #getting the percent difference 


        max_percent=np.max(percent_table) #getting the max of the prices
        index=np.where(percent_table == max_percent) #getting the position of the max to get the name
        index_n=name_list[index[0][0]] #finding the name from the index of the max
        print(max_percent*100-100, ' percent for ',index_n)

        arr_old_1=arr_new_1 #setting the new prices to compare 
        

        if max_percent>percent:  #Buying order if the price change more than the set percent
            
            #send order here with index_name

            print("order sent ------------")
            
            start=float(b.prices()[index_n]) #starting price from index name
            
            stoploss=net_risk*start #getting the price for the stoploss
            trail=start-stoploss #trailling stop value (difference between price and stoploss)

            high=start #to start the loop
            price_to_retain=start #to the start the loop
            
            end=0 #empty variable

            while price_to_retain>0:  #will always be above 0, can also use while True // Loop as before

                if high<float(b.prices()[index_n]): #if price higher than high, set new high from price
                    high=float(b.prices()[index_n]) ; print("new high :", high)
                    stop=high-trail #computing the new stop

                if float(b.prices()[index_n])<high-trail: #if price under high-trail, close the trade
                    end=float(b.prices()[index_n]); print("order closed")
                    #add close order here

                if float(b.prices()[index_n])<stoploss: #if price under stoploss price, close the trade
                    end=float(b.prices()[index_n]) ; print("order closed")

                t.sleep(1) #price refreshing rate, in seconds

                price_to_retain=float(b.prices()[index_n]) #actualizing price to retain

                print('high:', high, ' price: ', price_to_retain, ' stop:', stoploss)

                if end>0: #breaking the loop and computing the result
                    result=end*(1-com)-start*(1+com) #comission to simulate the spread
                    print('You won =', result)
                    break

Loop started :)
Searching....
2.893634005188588  percent for  BOTBTC
Searching....
2.0  percent for  IOSTBTC
Searching....
4.761904761904773  percent for  STMXBTC
Searching....
4.761904761904773  percent for  SCBTC
Searching....
1.5564202334630295  percent for  PHBTUSD
Searching....
7.692307692307708  percent for  DREPBTC
order sent ------------
high: 1.4e-07  price:  1.4e-07  stop: 1.39972e-07
high: 1.4e-07  price:  1.3e-07  stop: 1.39972e-07
order closed
order closed
high: 1.4e-07  price:  1.3e-07  stop: 1.39972e-07
You won = -1.0269999999999972e-08
Searching....
4.761904761904773  percent for  STMXBTC
Searching....
16.66666666666667  percent for  TNBBTC
order sent ------------
high: 7e-08  price:  7e-08  stop: 6.9986e-08
order closed
order closed
high: 7e-08  price:  6e-08  stop: 6.9986e-08
You won = -1.0129999999999999e-08
Searching....


KeyboardInterrupt: 

## 3. TESTING 


### 3.1 Old data

In [ ]:
klines = klines("BNBBTC", KLINE_INTERVAL_1MINUTE, "1 day ago UTC") #getkline

# feed it in the trade with sleep function to get the price change

### 3.2 New Data

## 4. OPTIMIZING

Running through historical data to find the best paramaters / optimizing Equity for the automatic trade